<a href="https://colab.research.google.com/github/ZJouba/Advent-of-Code-2022/blob/main/Day9/Day9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [185]:
import requests
import numpy as np

# file = requests.get('https://raw.githubusercontent.com/ZJouba/Advent-of-Code-2022/main/Day9/Day9.txt')
# input = [[a.split(' ')[0], int(a.split(' ')[1])] for a in list(filter(None, file.text.split('\n')))]

file = '''R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20'''
input = [[a.split(' ')[0], int(a.split(' ')[1])] for a in list(filter(None, file.split('\n')))]

print(input)

[['R', 5], ['U', 8], ['L', 8], ['D', 3], ['R', 17], ['D', 10], ['L', 25], ['U', 20]]


In [130]:
import numpy as np
import sys
np.set_printoptions(threshold=np.inf, linewidth=500)
from itertools import product

def travel(steps, current, direction):
  moves = []
  for step in range(steps):
    current = list(sum(x) for x in zip(current, direction))
    moves.append(current)
    # print(current)
  return moves, current

coordinates = []
currentCoord = (0,0)
for move in input:
  direction = move[0]
  if move[0] == 'L':
    moves, currentCoord = travel(move[1], currentCoord, (0, -1))
    coordinates.extend(moves)
  elif move[0] == 'U':
    moves, currentCoord = travel(move[1], currentCoord, (1, 0))
    coordinates.extend(moves)
  elif move[0] == 'R':
    moves, currentCoord = travel(move[1], currentCoord, (0, 1))
    coordinates.extend(moves)
  elif move[0] == 'D':
    moves, currentCoord = travel(move[1], currentCoord, (-1, 0))
    coordinates.extend(moves)

In [125]:
all_moves = list(product([-1,0,1], repeat=2))

def neighbors(coord):
  choices =  np.array(coord) + np.array(all_moves)
  choices = [item.tolist() for item in choices]
  return choices

In [126]:
# print(coordinates)
shape = np.max(coordinates, 0)+1
map = np.tile('.', shape*2)

tail = [0, 0]
tailSet = set()
for index, step in enumerate(coordinates):
  # print(f'Step: {step}')
  # print(f'Neighbors: {list(neighbors(step))}')
  if not tail in list(neighbors(step)):
    tail = coordinates[index-1]
  tailSet.add(tuple(tail))
  # map[tail[0]+shape[0], tail[1]+shape[1]] = '#'
  # print(f'Tail is: {tail}')
  # print('-'*5)

# map[shape[0], shape[1]] = 'S'

# print(np.flipud(map))
# print(np.count_nonzero(map == '#'))
print(len(tailSet))

6037


In [179]:
from copy import deepcopy

def update_position(position, offset):
  pos_y, pos_x = position
  offset_y, offset_x = offset
  return [offset_y+pos_y, offset_x+pos_x]

def update_tail(tail, head):
  offset_row = head[0] - tail[0]
  offset_col = head[1] - tail[1]

  if (abs(offset_row) <= 1 & abs(offset_col) <= 1): return tail
  elif (offset_row != 0 & offset_col != 0): move = [1 if offset_row > 0 else -1, 1 if offset_col > 0 else -1]
  else: 
    if offset_row > 0: move = [1, 0]
    if offset_row < 0: move = [-1, 0]
    if offset_col > 0: move = [0, 1]
    if offset_col < 0: move = [0, -1]

  return update_position(tail, move)

shape2 = np.max(coordinates, 0)+1
map2 = np.tile('.', shape2*2)

rope = [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]
tailSet = set()
for index, step in enumerate(coordinates[:13]):
  map2 = np.tile('.', shape2*2)
  newRope = [step, *rope]

  for idx in range(1, len(newRope)):
    newRope[idx] = update_tail(newRope[idx], rope[idx-1])
    map2[newRope[idx][0]+shape2[0], newRope[idx][0]+shape2[0]] = idx;
  rope = newRope
  
  for pos in reversed(range(len(rope))):
    knot = rope[pos]
    map2[knot[0]+shape2[0], knot[1]+shape2[1]] = pos
  print(np.flipud(map2))
  # print(f'Knots = {knots}')
  
  # prev = deepcopy(rope)
  # tailSet.add(tuple(knots[-1]))

print(len(tailSet))


RRRRRUUUUUUUULLLLLLLLDDDRRRRRRRRRRRRRRRRRDDDDDDDDDDLLLLLLLLLLLLLLLLLLLLLLLLLUUUUUUUUUUUUUUUUUUUU


36

In [187]:
moves = ''.join([char*n for char,n in input])
shape2 = np.max(coordinates, 0)+1

def update_pos(P, off):
    Py, Px = P
    Oy, Ox = off
    return (Oy+Py, Ox+Px)

def update_tail(T, H):
    off_row = H[0] - T[0]
    off_col = H[1] - T[1]

    if (abs(off_row)<=1) & (abs(off_col)<=1): return T

    elif (off_row!=0) and (off_col!=0):
        diag_move = 1 if off_row>0 else -1, 1 if off_col>0 else -1
        return update_pos(T, diag_move)

    else: 
        direction = (
            ('U' if off_row > 0 else 'D')
                if off_row!=0 else 
            ('R' if off_col > 0 else 'L')
            )
        return update_pos(T, dmap[direction])

dmap = {'L':(0,-1), 'R':(0,1), 'U':(1,0), 'D':(-1,0)}

old_rope = tuple([(0,0) for _ in range(10)])
history = [old_rope]

for move in moves:
    map2 = np.tile('.', shape2*2)
    new_rope = []
    new = update_pos(old_rope[0], dmap[move])
    new_rope.append(new)
    map2[new[0]+shape2[0], new[1]+shape2[1]] = 'H';
    for idx in range(1, 10):
        new = update_tail(old_rope[idx], new_rope[idx-1])
        new_rope.append(new)
        map2[new[0]+shape2[0], new[1]+shape2[1]] = idx;
    history.append(tuple(new_rope))
    old_rope = new_rope
    print(np.flipud(map2))

len(set([t[-1] for t in history]))

[['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.

36